In [ ]:
import pandas as pd
from utils import (
    getSharesIdx,
    getSharesIdxCI95,
    link_dt,
    load_trip_assign_day,
    out_dir,
    prep_data_1d,
    prep_data_2d,
    tour_extract_allwk_dir,
    tour_extract_wkday_dir,
    write_to_excel,
)
from xlsxwriter.utility import xl_rowcol_to_cell


In [ ]:
out_filepath = out_dir / "01_AllTrips_Mode.xlsx"

In [ ]:
## Process weekday only trip records
trip = load_trip_assign_day(
    tour_extract_wkday_dir / "trip-assign_day.csv", depart_hour=False
).to_pandas()
trip = link_dt(trip)

In [ ]:
col_dict = {
    "mode": {
        "desc": "Mode",
        "col": "mode",
        "vals": range(1, 10),
        "labels": [
            "1-walk",
            "2-bike",
            "3-DA",
            "4-HOV2",
            "5-HOV3",
            "6-walktransit",
            "7-drivetransit",
            "8-schoolbus",
            "9-TNC (inc taxi)",
        ],
    },
    "day": {
        "desc": "DOW",
        "col": "day",
        "vals": [7, 8],
        "labels": ["6_Sat", "7_Sun"],
    },
}

In [ ]:
writer = pd.ExcelWriter(out_filepath, engine="xlsxwriter")
workbook = writer.book
# format1 = workbook.add_format({"num_format": "#,##0.0"})

In [ ]:
row = 0
sname = "Weekday"
d1_dict = col_dict["mode"]

title = "All Trips by " + d1_dict["desc"]
tab = prep_data_1d(
    trip,
    d1_dict["desc"],
    d1_dict["col"],
    "trexpfac",
    d1_dict["vals"],
    d1_dict["labels"],
)
row = write_to_excel(writer, tab, sname, title, row)

title = "Column Shares by " + d1_dict["desc"]
row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)

tab2 = tab.copy()
tab2.iloc[-1, -1] = tab2.iloc[-1, 0]

title = "Column Shares 95% CI by " + d1_dict["desc"]
row = write_to_excel(writer, getSharesIdxCI95(tab.copy()), sname, title, row)

tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
_ = writer.sheets[sname].set_column(tab_range, 11)  # , format1)

In [ ]:
# Process all week trip records
trip = load_trip_assign_day(tour_extract_allwk_dir / "trip-assign_day.csv").to_pandas()
trip = link_dt(trip)

In [ ]:
row = 0
sname = "Weekend"
d1_dict = col_dict["mode"]
d2_dict = col_dict["day"]

wt_cols = ["count", "trexpfac"]
wt_desc = ["(Unweighted)", "(Weighted)"]

for wc, wd in zip(wt_cols, wt_desc):
    title = "All Trips by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
    tab = prep_data_2d(
        trip,
        d1_dict["col"],
        d1_dict["vals"],
        d1_dict["labels"],
        d2_dict["col"],
        d2_dict["vals"],
        d2_dict["labels"],
        wc,
    )
    row = write_to_excel(writer, tab.astype("float64"), sname, title, row)

    if wc == "count":
        tab2 = tab.copy()
    else:
        tab2.iloc[:-1, :-1] = tab.iloc[:-1, :-1]

    title = "Column Shares by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
    row = write_to_excel(writer, getSharesIdx(tab.copy()), sname, title, row)

    title = (
        "Column Shares 95% CI by "
        + d2_dict["desc"]
        + " and "
        + d1_dict["desc"]
        + " "
        + wd
    )
    row = write_to_excel(writer, getSharesIdxCI95(tab2.copy()), sname, title, row)
tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
_ = writer.sheets[sname].set_column(tab_range, 11)  # , format1)

In [ ]:
writer.close()